# 🦜🔗 Langchain Demo

Welcome! This example will give you a first look at langchain. 

In [ ]:
%pip install -r requirements.txt

## Load Environment

Load ENV Variables from .env file

needed ENV vars:

```
# OpenAI Hosted Variante
OPENAI_API_KEY=<required>
OPENAI_ORGANIZATION=""
OPENAI_MODEL="gpt-4-0125-preview"

# Azure Hosted Variante
AZURE_OPENAI_API_KEY=<required>
AZURE_OPENAI_ENDPOINT=<required>
AZURE_OPENAI_API_VERSION="2023-12-01-preview"
AZURE_OPENAI_DEPLOYMENT_NAME=<required>
``` 

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

## Erster Test

### OpenAI-Modelle
ChatGPT oder auch jedes andere LLM benutzen ist relativ einfach mit Langchain

In diesen Test nutzen wir das neueste "gpt-4-turbo" Model - mögliche Large Language Modelle von OpenAI sind:
- `gpt-35-turbo`  Das günstigste und am weitesten verbreitete Modell
- `gpt-4`  Das neue und bessere GPT Modell
- `gpt-4-turbo`  Turbo-variante von gpt-4 (günstiger, schneller, kleinere maximale Länge des Text-Outputs)
- `gpt-4-vision`  Ein "multimodales" Modell, welches auch auf Bilder trainiert wurde.

OpenAI trainiert diese Versionen laufend neu, was dazu führen kann, das Anfragen an das LLM plötzlich andere Antworten geben.
Möchte man dies verhindern, kann man seine Applikation auf einen Snapshot (z.b. gpt-4-0613) festsetzen.
Dies ist insbesondere wichtig, wenn die Applikation vom Output des LLM bestimmte Strukturen erwartet, beispielsweise eine bestimmte XML-Syntax o.Ä.

OpenAI-Modelle werden nicht nur von OpenAI selbst gehostet, sondern auch von Azure.
Diese muss man auf dem Azure Portal selbst als Endpunkte konfigurieren, in der Regel leiden die OpenAI Azure Deployments weniger unter hoher Auslastung

### Andere Modelle
Auch wenn wir nicht damit arbeiten werden, ist es vielleicht ganz gut, die Namen der "großen" Konkurrenz-Modelle einmal gehört zu haben:
- `Gemini` Das neueste Google-Modell. Es hat den Fokus insbesondere auf multimodalem Input.
- `Claude` Claude ist die LLM-Reihe von Anthropic. Sehr viel Instruction-Tuning.
- `Mixtral` Das aktuell beste Open-Source Modell. Entwickelt von Mistral AI. Ein guter Kandidat für ein selbst gehostetes LLM.

### Temperatur
Alle LLMs sind nicht deterministisch. Aber die Temperatur ist ein Parameter, mit der man die Variabilität von Antworten hoch und runter schrauben kann.
Wie bei normalen Atomen ist die Bewegung niedrig, wenn die Temperatur niedrig ist. Wenn man die Temperatur hochschraubt, wird viel gewackelt.
Der Temperatur-Parameter ist üblicherweise ein Fließkommawert zwischen 0 und 1.

### Streaming
Nicht alle LLMs bieten die Möglichkeit, Token für Token live zu streamen. OpenAI-Modelle können es, man kann dies mit dem Streaming-Parameter einstellen.

### Links:
- https://www.langchain.com/
- https://python.langchain.com/docs/get_started/introduction
- https://platform.openai.com/docs/models/gpt-3-5
- https://platform.openai.com/docs/models/gpt-4
- https://platform.openai.com/docs/deprecations
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

#### Wir definieren das LLM

In [ ]:
# Der Folgende Code-Abschnitt ist nicht wirklich dafür gedacht, sofort verstanden zu werden. Da steckt etwas Langchain-Magie drin.
# Er sollte bestenfalls überflogen oder einfach nicht beachtet werden.
# Man sollte sich aber merken, was er tut, um bei Bedarf hierher zurück zu springen um Code zu kopieren:
# Definiere ein LLM, das zur Laufzeit konfiguriert werden kann (OpenAI, AzureOpenAI, Temperatur....)
# Diese Langchain-Funktionalität wird sich beim Programmieren später einmal sehr nützlich erweisen.
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langchain_core.runnables import ConfigurableField
openai_llm = ChatOpenAI(model=os.environ["OPENAI_MODEL"]).configurable_fields(temperature=ConfigurableField(id="temperature", is_shared=True))
azure_llm = AzureChatOpenAI(azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]).configurable_fields(temperature=ConfigurableField(id="temperature", is_shared=True))
llm = openai_llm.configurable_alternatives(ConfigurableField(id="llm"), default_key="openai", azure=azure_llm)

#### Wir probieren aus:

In [ ]:
print(llm.with_config(configurable={"llm": "openai"}).invoke("Hi OpenAI! Kannst Du mir gerade mal einen hessischen Trinkspruch auf den Taunus im Dialekt erzeugen?").content)

#### Jetzt nochmal mit Streaming. Dazu rufen wir nicht invoke sondern astream auf (a für async). Wir drehen etwas an der Temperatur, damit die Ergebnisse spannend bleiben

In [ ]:
chunks = []
async for chunk in llm.with_config(configurable={"llm": "openai", "temperature": 1}).astream("Erklär mir in einem Satz Quantenmechanik."):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

## Token

Token sind die kleinste Einheit des LLM. Das haben wir gerade beim Streaming schön gesehen. Der Stream kommt Token für Token aus dem LLM gepurzelt.

Das LLM rechnet aus der Eingabe und den bisher errechneten Token die Wahrscheinlichkeit für den nächsten Token aus. Dieser neue Token wird dann angehängt und der nächste Token wird ermittelt.

So geht das immer weiter. Bis der nächste wahrscheinlichste Token ein Stop-Zeichen ist. Auf diese Weise generieren LLMs die wahrscheinlichste Fortführung der Eingabetoken.

Token können Wörter, machmal sogar Wortgruppen oder auch nur einzelne oder mehrere Buchstaben sein.

Die Bepreisung der LLMs ist an die Tokenanzahl (Eingabe und Ausgabe) gekoppelt.


Links:
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

tokens = encoding.encode("AI ist eine tolle Sache.")

decoded_tokens = [encoding.decode_single_token_bytes(token).decode('utf-8') for token in tokens]
for token in decoded_tokens:
    print(token)

## Prompt Engineering und Templates in Langchain

Um die Dinge von der AI zu bekommen, die man erwartet, stellt man am besten sehr konkrete und präzise Anfragen.

Weil eine AI oft an ein bestimmtes Feld von Aufgaben gekoppelt ist, gibt man die Rahmenanweisung dann in ein Template ein, um nicht immer wieder die gleiche Rahmenanweisung zu schreiben.

Die jeweilige konkrete Nutzeranfrage wird dann in das Template eingefügt und das ausgefüllte Template ans LLM übergeben.

Der Trend geht immer mehr zu Chat-Modellen. Hierbei ist die Information, die man dem LLM gibt, in "Messages" unterteilt. Besondere Gewichtung hat eine System-Message. Diese kann Rahmenanweisungen enthalten, an die sich das LLM halten soll. Dem Nutzer wird es schwer fallen, das LLM dazu zu bewegen, sich über eine Anweisung in der System-Message hinweg zu setzen. Das LLM wurde ganz einfach darauf trainiert, sich an die Anweisungen einer System-Message strikt zu halten.

### Links
- https://python.langchain.com/docs/get_started/quickstart#prompt-templates
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering
- https://learnprompting.org/docs/intro
- https://www.promptingguide.ai/
- https://smith.langchain.com/hub

In [ ]:
from langchain.prompts import  ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist ein {beruf} aus Frankfurt."),
        ("human", "Erklär in 2 Sätzen im hessischen dialekt warum Deine Kunden aus {ort} die besten sind."),
    ]
)

print(prompt.format(beruf="Bäcker", ort="Bad Homburg"))


### Langchain Hub Beispiel

Weil das "Prompt-Engineering" ein bisschen Übung braucht und es diverse Tricks gibt, hat LangChain einen "Hub", auf dem man eine ganze Reihe vorgefertigter Prompts für verschiedene Anwendungsfälle findet.

Dort kann man sich inspirieren lassen, Prompts forken oder auch selbst etwas für andere Leute zur Verfügung stellen, wenn es sich als nützlich erweist.

Links:
- https://smith.langchain.com/hub/borislove/customer-sentiment-analysis

In [ ]:
from langchain import hub
sentiment_prompt = hub.pull("borislove/customer-sentiment-analysis")

client_letter="""Ich bin von dem Volleyballschläger zutiefst enttäuscht. Zuerst ist der Griff abgefallen, danach auch noch der Dynamo. Außerdem riecht er noch schlechter als er schmeckt. Wieso ist das immer so ein Ärger mit euch?"""
format_instructions="""Zusätlich zur numerischen Klassifizierung sollst du herausfinden, was der Kunde gerne gehabt hätte. Antworte auf deutsch."""

print(sentiment_prompt.format(client_letter = client_letter, format_instructions = format_instructions))

## Jetzt fängt es an, etwas technischer zu werden. Wieso heißt LangChain eigentlich LangChain?

Langchain definiert einige Python-Operatoren neu, wenn sie zwischen LangChain-Objekten stehen. Der bekannteste ist die Pipe: |

Wenn die Pipe zwischen zwei Langchain-Objekten steht, wird die Ausgabe des ersten Obekts an das nächste weitergegeben. Damit erhält man eine "Chain" von "Runnables"

#### Links
- https://python.langchain.com/docs/modules/chains

In [ ]:
from langchain.schema import StrOutputParser # Hilft beim Formatieren
chain = prompt | llm | StrOutputParser()
print(chain.invoke({"beruf":"Schlachter", "ort":"Hanau"}))

In [ ]:
# Streaming
async for chunk in chain.with_config(configurable={"llm": "openai"}).astream({"beruf":"Bäcker", "ort":"Würzburg"}):
    print(chunk, end="", flush=True)

In [ ]:
# Funktioniert auch das Beispiel vom Hub?
sentiment_chain = sentiment_prompt | llm | StrOutputParser()
async for chunk in sentiment_chain.astream({"client_letter" :client_letter, "format_instructions" : format_instructions}):
    print(chunk, end="", flush=True)

In [ ]:
# Wir können dynamisch die format_instructions des Templates überschreiben, um neue Ergebnisse zu bekommen
format_instructions="""Zusätlich zur sentiment Analysis ist es deine Aufgabe, die Sinnhaftigkeit der Kundenäußerung zu überprüfen."""
async for chunk in sentiment_chain.astream({"client_letter" :client_letter, "format_instructions" : format_instructions}):
    print(chunk, end="", flush=True)

## Embeddings (Textembeddings, Vektoren)

#### Embeddings sind ein ganz grundlegender Baustein von LLM-basierten Applikationen. Daher wollen wir hier noch einmal so kurz wie möglich aber so umfassend wie nötig das Konzept Embedding erklären.

Beim Embedden wird ein Textstück genommen und von einem einen ML-Modell in einen hochdimensionalen Vektor umgewandelt (eigentlich: auf einen Vektor projeziert).

Es gibt unterschiedliche Methoden, wie dieser Vektor erstellt wird und die genaue Art und Weise, wie der Embeddingprozess vonstatten geht, ist eine kleine Wissenschaft für sich.

**Allerdings haben alle Text-Embeddings folgende Eigenschaft: Wenn zwei Textstücke ähnlichen Inhalt haben, liegen die resultierenden Vektoren im Raum nahe beieinander.**

**Man erhält durch den Embeddingprozess also eine Möglichkeit, sehr viele (!) Textstücke, zusammen mit der Information, wie ähnlich diese Texte inhaltlich sind, abzuspeichern!**

Vektoren (i.e. Embeddings) können vom Computer sehr schnell verarbeitet werden. Man kann damit dann so schöne Dinge tun wie:
- Suche (wobei die Ergebnisse nach semantischer Relevanz geordnet werden)
- Clustering (wobei Textzeichenfolgen nach Ähnlichkeit gruppiert werden)
- Empfehlungen (wobei Elemente mit zugehörigen Textzeichenfolgen empfohlen werden)
- Anomalieerkennung (wobei Ausreißer mit geringem Zusammenhang identifiziert werden)
- Klassifizierung (wobei Textzeichenfolgen nach ihrer ähnlichsten Bezeichnung klassifiziert werden)

Die Standard-Suchanwendung läuft ab wie folgt:
- Alle Dokumente, die eine Wissensdatenbank bilden sollen, (z.B. Betriebsanleitungen eines Maschinenherstellers) werden **im Vorhinein** vektorisiert.
- Dann stellt ein Nutzer eine Anfrage an die KI (bezüglich einer Maschine des Maschinenherstellers).
- Diese Frage wird ebenfalls vektorisiert (das geht sehr schnell).
- Dann wird in den Vektoren der Betriebsanleitungen nach Vektoren gesucht, die der Frage semantisch ähnlich sind (geht auch sehr schnell). Bei einem guten Embedding liegen Frage-Antwort-Paare im Vektorraum nahe beieinander.
- Die ähnlichsten Vektoren werden rückaufgelöst (d.h. man sieht nach, welche ursprünglichen Dokumente hinter den Vektoren stehen).
- Diese Dokumente werden dann der AI als Kontext zum Beantworten der Frage mitzugeben.

Embeddings machen dies möglich, weil sie nicht auf der Grundlage von Zeichenfolgen arbeiten sondern wirklich eine semantische Nähe zueinander finden. Die Wörter "König" und "Prinz" sind sich im Vektorraum z.B. sehr ähnlich, obwohl die Buchstabenfolge sehr unterschiedlich ist.

Es gibt sehr viele Embedding-Modelle, die für alle möglichen Fälle optimiert sind.
Modelle können einpsrachig oder mehrsprachig sein, wobei man für die Mehrsprachigkeit einen Qualitätsverlust in Kauf nehmen muss (siehe weiter unten)!
Es gibt multimodale Embeddings die z.B. für das Wort Schraube und das Bild einer Schraube sehr ähnliche Vektoren herausgeben.

Links:
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- https://python.langchain.com/docs/integrations/text_embedding/azureopenai
- https://app.twelvelabs.io/blog/multimodal-embeddings

#### Embeddings in der Praxis

Wir benutzen die Endpunkte von OpenAI für den Embeddingprozess. Natürlich kann man lokal ein kleines Embedding-Modell betreiben, wenn man dies aus Gründen tun möchte.
OpenAI kümmert sich darum, dass das Embedding-Modell gut trainiert ist. Diese Modelle werden auch laufend besser und es entstehen neue Ausdifferenzierungen für Spezialfälle.
Implizit wird beim Aufruf embeddings.embed_query() also der Access-Token aus der .env gelesen und wir bezahlen eine kleine Summe dafür, dass OpenAI uns Embedding-Prozess abnimmt.

In [ ]:
# Der Folgende Code-Abschnitt ist nicht wirklich dafür gedacht, sofort verstanden zu werden. Da steckt eine ganze Menge Langchain-Magie drin.
# Er sollte bestenfalls überflogen oder einfach nicht beachtet werden.
# Man sollte sich merken, was er tut, um bei Bedarf hierher zurück zu springen um Code zu kopieren:
# Baue eine Langchain-Funktion, die eine definierte Konfigurationsschnittstelle zur Laufzeit mit dem Nutzer bietet.
# Die Schnittstelle kümmert sich außerdem um Integrität!
# Hier kann ein Nutzer eine Funktion aufrufen, die vektorisiert und das Embedding-Modell selbst wählen.

from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain_core.runnables import (
    ConfigurableFieldSingleOption,
    RunnableBinding,
    RunnableConfig,
    chain,
)
load_dotenv()

class EmbedText(RunnableBinding):
    embeddings: str

    def __init__(
        self, embeddings: str = "openai", config: RunnableConfig = None, **kwargs
    ):
        @chain
        def _embed_text(text: str):
            if self.embeddings == "openai":
                _embeddings = OpenAIEmbeddings()
            elif self.embeddings == "azure":
                _embeddings = AzureOpenAIEmbeddings(
                    azure_deployment="textembeddingada002"
                )
            return _embeddings.embed_query(text)

        kwargs.pop("bound", None)
        super().__init__(
            embeddings=embeddings, bound=_embed_text, config=config, **kwargs
        )


embed_text = EmbedText().configurable_fields(
    embeddings=ConfigurableFieldSingleOption(
        id="embeddings",
        default="openai",
        options={"openai": "openai", "azure": "azure"},
    )
)

embed_text.config_schema().schema().get("definitions")

In [ ]:
# Ein Beispieltext
text = "This is a test document."

# Hier configurieren wir, welches Embedding wir haben möchten.
query_result = embed_text.with_config(configurable={"embeddings": "azure"}).invoke(
    text
)

# Wie viele Dimensionen hat so ein Vektor?
print("Dimensions: ",len(query_result))

# Und wie sieht er aus? Zeige die ersten drei Einträge.
print(query_result[:3])

### Veranschaulichung von semantischer Nähe mittels Abstandsberechnung der zugehörigen Vektoren.
Kleinere Zahl ≙ Näher an der Referenz

In [ ]:
from langchain.evaluation import load_evaluator
import pandas as pd
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from ipydatagrid import DataGrid, BarRenderer
from bqplot import LinearScale, ColorScale

# Wir machen uns nicht die Mühe, für das Plotten eine konfigurierbare Langchain-Funktion zu bauen...
def get_embeddings(e):
    if e == "openai":
        return OpenAIEmbeddings()
    elif e == "azure":
        return AzureOpenAIEmbeddings(azure_deployment="textembeddingada002")

# Unser Referenzwort
reference = 'ICE'

# Wie nahe sind diese Worte an der Referenz?
test_set = [ 'Regionalbahn',
              'S-Bahn',
              'Nachtzug',
              'Flugzeug',
              'Icecream',
              'Zeppelin',
            ]

# Langchain tut die Arbeit für uns...
evaluator = load_evaluator("embedding_distance", embeddings=get_embeddings("openai"))
distances = []
for item in test_set:
    distance = evaluator.evaluate_strings(prediction=item, reference=reference)
    distances.append(distance["score"])

# Als Tabelle ausdrucken
df = pd.DataFrame({
    "Wort": test_set,
    "Entfernung": distances
})
renderers = {
 "Entfernung": BarRenderer(
        horizontal_alignment="center",
        bar_color=ColorScale(min=0, max=1, scheme="viridis"),
        bar_value=LinearScale(min=0, max=1),
    )
}
grid = DataGrid(df, base_column_size=250, renderers=renderers)
grid.transform(
    [
        {"type": "sort", "columnIndex": 2, "desc": False},
    ]
)
grid

### Debug Informationen gewünscht?

In [ ]:
from langchain.globals import set_debug, set_verbose

In [ ]:
## Und jetzt selber mal Ausprobieren

In [ ]:
print(chain.run({
    'beruf': "Programmierer",
    'ort': "Stuttgart"
    }))